<a href="https://colab.research.google.com/github/kisung2577/CMPE-255/blob/main/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Decision Tree from Scratch
A decision tree is a flowchart-like structure in which each internal node represents a "test" on an attribute (e.g. whether a coin flip comes up heads or tails), each branch represents the outcome of the test, and each leaf node represents a class label (decision taken after computing all attributes). The paths from root to leaf represent classification rules.

In decision analysis, a decision tree and the closely related influence diagram are used as a visual and analytical decision support tool, where the expected values (or expected utility) of competing alternatives are calculated.

A decision tree consists of three types of nodes:

Decision nodes – typically represented by squares
Chance nodes – typically represented by circles
End nodes – typically represented by triangles


##Necessary Imports

In [ ]:
import pandas as pd
import numpy as np


##Mount Google drive to load data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##Load data of bank note authentication http://archive.ics.uci.edu/ml/datasets/banknote+authentication

In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/255DataMining/dataset/data_banknote_authentication.txt',
                   names = ['variance','skewness', 'kurtosis', 'entropy', 'class'])
data

,variance,skewness,kurtosis,entropy,class
0,3.62160,8.66610,-2.8073,-0.44699,0
1,4.54590,8.16740,-2.4586,-1.46210,0
2,3.86600,-2.63830,1.9242,0.10645,0
3,3.45660,9.52280,-4.0112,-3.59440,0
4,0.32924,-4.45520,4.5718,-0.98880,0
...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,1
1368,-1.38870,-4.87730,6.4774,0.34179,1
1369,-3.75030,-13.45860,17.5932,-2.77710,1
1370,-3.56370,-8.38270,12.3930,-1.28230,1


In [ ]:
X = data.drop('class', axis=1).to_numpy()
y = data['class'].to_numpy()

##Gini Impurity
The Gini impurity measure is one of the methods used in decision tree algorithms to decide the optimal split from a root node, and subsequent splits. 
Gini Impurity tells us what is the probability of misclassifying an observation. Note that the lower the Gini the better the split.

Function that returns the gini impurity in a given list of classes

In [ ]:
def gini_impurity(y):
    y = np.asarray(y)
    m = y.shape[0]
    classes = np.unique(y)
    g_i = np.asarray([(np.sum(y == c)/m) for c in classes])
    G = 1 - np.sum(g_i**2)
    return G

Function that returns the weighted gini impurity of a node

In [ ]:
def get_gini(G_left, G_right):
    m_l = G_left.size
    m_r = G_right.size
    G = (m_l/(m_l+m_r))*G_left + (m_r/(m_l+m_r))*G_right
    return G

Function to split a node X, into two nodes, based on a feature and a threshold for that feature

In [ ]:
def split_node(X, y, feature, threshold):
    X_left, y_left, X_right, y_right = [], [], [], []
    for (row,y_i) in zip(X,y):
        if row[feature] < threshold:
            X_left.append(row)
            y_left.append(y_i)
        else:
            X_right.append(row)
            y_right.append(y_i)
    return X_left, y_left, X_right, y_right

Function to select the best threshold and feature for a given node given by X

In [ ]:
def best_split(X, y):
    X = np.asarray(X)
    y = np.asarray(y)
    best_feature, best_threshold, best_G, best_children = 0, 0, 2, 0
    for feature in range(X.shape[1]):
        for row in X:
            X_left, y_left, X_right, y_right = split_node(X, y, feature, row[feature])
            G_left  = gini_impurity(y_left)
            G_right = gini_impurity(y_right)
            G = get_gini(G_left, G_right)
            if G < best_G:
                Node_left = [X_left, y_left]
                Node_right = [X_right, y_right]
                best_feature, best_threshold, best_G, best_children = \
                feature, row[feature], G, [Node_left, Node_right]
    return dict({'feature':best_feature, 'threshold':best_threshold, 'children':best_children})

##Tree structure
Below tree will consist of nodes.
Each node will be of the form [Node_left, Node_right] accessible through the 'children' key of the dictionary returned by best_split.
Both Node_left and Node_right will be of the form [X, y]

Dummy data to test

In [ ]:
Xd = np.asarray([[2.771244718,1.784783929],
                [1.728571309,1.169761413],
                [3.678319846,2.81281357],
                [3.961043357,2.61995032],
                [2.999208922,2.209014212],
                [7.497545867,3.162953546],
                [9.00220326,3.339047188],
                [7.444542326,0.476683375],
                [10.12493903,3.234550982],
                [6.642287351,3.319983761]])
yd = np.asarray([0,0,0,0,0,1,1,1,1,1])

Function that converts a node (node_left or node_right) to a leaf

In [ ]:
def leaf(Node):
    X, y = Node
    return np.argmax(np.bincount(y))

Function to split nodes recursively

In [ ]:
def tree(Node, max_depth, min_size, depth): 
    
    left, right = Node['children']
    # No need for children anymore :)
    del(Node['children'])
    
    # BASE CASES:
    ## If no children:
    if not left or not right:
        Node['left'] = Node['right'] = leaf(left + right)
        return
    ## Checking whether max depth has been reached:
    if depth >= max_depth:
        Node['left'], Node['right'] = leaf(left), leaf(right)
        return
    
    # RECURSION:
    ## Left child node: left[1] yields y_left, which a list.
    if len(left[0]) <= min_size:
        Node['left'] = leaf(left)
    else:
        Node['left'] = best_split(left[0], left[1])
        tree(Node['left'], max_depth, min_size, depth+1)
        
    ## Process right child
    if len(right[1]) <= min_size:
        Node['right'] = leaf(right)
    else:
        Node['right'] = best_split(right[0], right[1])
        tree(Node['right'], max_depth, min_size, depth+1)

Function to build the decision tree

In [ ]:
def build_tree(X, y, max_depth, min_size):
    root = best_split(X, y)
    tree(root, max_depth, min_size, 1)
    return root

Function that prints the tree

In [ ]:
def print_tree(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            print_tree(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))

Function to make a decision with the tree through tree traversal

In [ ]:
def predict(Node, row):
    # Recursively traverse the tree
    if row[Node['feature']] < Node['threshold']:
        if isinstance(Node['left'], dict):
            # Go left
            return predict(Node['left'], row)
        else:
            # Base case reached, return the leaf
            return Node['left']
    else:
        if isinstance(Node['right'], dict):
            # Go right
            return predict(Node['right'], row)
        else:
            # Base case reached, return the leaf
            return Node['right']

Classification and Regression Tree Algorithm (CART)

In [ ]:
def cart(X_train, y_train, X_test, y_test, max_depth, min_size):
    tree = build_tree(X_train, y_train, max_depth, min_size)
    predictions = []
    for row in X_test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

In [ ]:
print_tree(build_tree(Xd, yd, 2, 1))

feature
	0
threshold
	6.642287351
left
	feature
		0
	threshold
		2.771244718
	left
		0
	right
		0
right
	feature
		0
	threshold
		7.497545867
	left
		1
	right
		1


In [ ]:
predictions = cart(Xd, yd, Xd, yd, 1, 1)
print(np.sum(predictions==yd)/yd.shape[0])

1.0


Testing on real data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
y_pred = cart(X_train, y_train, X_test, y_test, 5, 10)
print(confusion_matrix(y_test, y_pred))
print(f1_score(y_test, y_pred))

[[134  54]
 [  5 150]]
0.8356545961002786
